# 과제 3: 서버리스 추론용으로 모델 배포

## 과제 3.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import sys
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

훈련 및 튜닝 실습에서 사용했던 모델을 기본 Amazon Simple Storage Service(Amazon S3) 버킷에 저장합니다. **create_model**을 사용하여 모델을 설정하고 훈련된 모델을 참조하도록 **ModelDataUrl**을 구성합니다.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## 과제 3.2: 실습에서 제공되는 합성 및 재훈련된 모델에서 엔드포인트 생성

Amazon SageMaker Serverless Inference는 기계 학습 모델을 배포하고 크기를 조정할 수 있는 목적별 추론 옵션입니다. 트래픽 급증 현상 간에 유휴 기간이 있으며 콜드 스타트가 허용되는 워크로드에는 Serverless Inference를 사용하는 것이 좋습니다. 서버리스 엔드포인트는 컴퓨팅 리소스를 자동으로 시작한 다음 트래픽에 따라 리소스를 확장 및 축소합니다. 그러므로 인스턴스 유형을 선택하거나 크기 조정 정책을 관리할 필요가 없습니다. 즉, 번거롭게 서버를 선택하고 관리하지 않아도 됩니다. Serverless Inference를 AWS Lambda와 통합하면 고가용성, 기본 제공 내결함성 및 자동 크기 조정 기능이 제공됩니다.

Amazon SageMaker Python SDK를 사용하여 서버리스 엔드포인트를 생성할 때는 다음의 3단계를 수행합니다. 이러한 단계는 실시간 엔드포인트에 사용되는 단계와 동일하지만 구성은 서로 다릅니다.
1. SageMaker에서 SageMaker 모델을 생성합니다.
2. HTTPS 엔드포인트용 엔드포인트 구성을 생성합니다.
3. HTTPS 엔드포인트를 생성합니다.

모델은 이미 생성했으므로 엔드포인트 구성과 엔드포인트를 생성할 수 있습니다. 

먼저 사용하려는 엔드포인트 구성 이름과 메모리 크기를 설정합니다. 그런 다음 CreateEndpointConfig API를 호출합니다.

엔드포인트 구성을 생성하려면 다음 옵션을 설정해야 합니다.
- **VariantName**: 프로덕션 변형(프로덕션 환경의 모델 하나 이상)의 이름입니다.
- **ModelName**: 호스팅할 모델의 이름입니다. 모델을 생성할 때 지정한 이름을 사용하면 됩니다.
- **ServerlessConfig**: 엔드포인트를 서버리스로 설정하는 옵션입니다. **MemorySizeInMB** 및 **MaxConcurrency**의 값을 구성합니다.
    - **MemorySizeInMB**: 할당되는 메모리 크기(1024, 2048, 3072, 4096, 5120 또는 6144MB)입니다.
    - **MaxConcurrency**: 동시 호출 수(1~200)입니다.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-serverless-endpoint-{}'.format(create_date)                              

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant
            "ServerlessConfig": {
                "MemorySizeInMB": 2048, # The memory size
                "MaxConcurrency": 20 # Number of concurrent invocations
            }
        } 
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

다음으로는 엔드포인트를 생성합니다. 서버리스 엔드포인트를 생성하면 SageMaker가 컴퓨팅 리소스를 자동으로 프로비저닝하고 관리합니다. 그러므로 엔드포인트에 대한 추론 요청을 실행하여 응답으로 모델 예측을 수신할 수 있습니다. SageMaker는 요청 트래픽을 처리할 수 있도록 필요에 따라 컴퓨팅 리소스를 확장 및 축소하므로 사용량만큼만 비용을 지불하면 됩니다.

SageMaker에서 제공하는 컨테이너를 선택할 수도 있고 직접 생성한 컨테이너를 사용할 수도 있습니다. 서버리스 엔드포인트의 최소 RAM 크기는 1024MB이고 최대 RAM 크기는 6144MB입니다. Serverless Inference는 사용자가 선택하는 메모리에 비례하도록 컴퓨팅 리소스를 자동 할당합니다.

엔드포인트가 작동 중이면 Helper 함수는 엔드포인트 Amazon 리소스 이름(ARN)을 인쇄합니다. 엔드포인트 생성을 실행하려면 약 5-7분이 소요됩니다.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


SageMaker Studio의 **Endpoints** 탭에서 엔드포인트 세부 정보를 확인할 수 있습니다.

다음 단계에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **serverless_inference_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **serverless_inference_ko_kr.ipynb** 탭을 선택한 다음, 도구 모음에서 **File**과 **New View for Notebook**을 선택합니다. 그러면 엔드포인트를 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 엔드포인트 탐색을 완료한 후 **serverless_inference_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. **SageMaker Home** 아이콘을 선택합니다.
2. **Deployments**를 선택해서 확장합니다.
3. **Endpoints**를 선택합니다.

SageMaker Studio에서 **Endpoints** 탭이 표시됩니다.

4. **Name** 열에서 **income-model-serverless-** 가 있는 엔드포인트를 선택합니다.

엔드포인트가 표시되지 않으면 엔드포인트가 나타날 때까지 새로 고침 아이콘을 선택합니다.

SageMaker Studio에서 **ENDPOINT DETAILS** 탭이 표시됩니다.

5. **AWS settings** 탭을 선택합니다.

생성이 완료되기 전에 엔드포인트를 열었다면 **Endpoint status**가 *Creating*에서 *InService*로 바뀔 때까지 새로 고침 아이콘을 선택합니다.

**Endpoint type**이 **Serverless**로 표시됩니다. **Endpoint runtime settings** 섹션에는 이전 과제를 수행할 때 노트북에서 선택한 구성이 표시됩니다.

## 과제 3.3: 고객 레코드를 사용하는 서버리스 추론을 위해 엔드포인트 호출

SageMaker 호스팅 서비스를 사용하여 모델을 배포한 후에는 엔드포인트로 테스트 데이터를 전송하여 해당 엔드포인트에서 모델을 테스트할 수 있습니다.

엔드포인트에서 트래픽이 장시간 수신되지 않다가 새 요청이 갑자기 수신되는 경우 해당 요청을 처리하기 위해 엔드포인트가 컴퓨팅 리소스를 스핀업하는 데 다소 시간이 걸릴 수도 있습니다. 이러한 현상을 일컬어 ‘콜드 스타트’라고 합니다. 서버리스 엔드포인트는 온디맨드 방식으로 컴퓨팅 리소스를 프로비저닝하므로 엔드포인트에서 콜드 스타트 현상이 발생할 수 있습니다. 동시 요청 수가 현재 동시 요청 사용량을 초과할 때도 콜드 스타트 현상이 발생할 수 있습니다. 콜드 스타트 시간은 모델 크기, 모델을 다운로드하는 데 걸리는 시간, 그리고 컨테이너 시작 시간에 따라 달라집니다.

서버리스 추론 및 콜드 스타트의 작동 방식에 관한 자세한 내용은 [서버리스 추론](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html)을 참조하세요.

고객 레코드가 몇 개 더 수신되었습니다. 소득 값이 1인 레코드와 소득 값이 0인 레코드를 한 세트로 사용하여 엔드포인트를 호출해 엔드포인트가 작동하는지를 확인합니다. 각 레코드의 예측 점수 목록이 출력됩니다. 

In [ ]:
#invoke-endpoint-serverless-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('47,0,4,9,0,3,4,0,1,0,1902,60\n' +
                '53,0,0,0,0,2,4,0,1,0,0,40\n' +
                '44,0,0,0,2,0,1,0,1,14344,0,40\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

start_time = time.time()
response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,1,2,1,0,0,0,35\n' +
                '56,2,1,1,0,1,0,0,0,0,0,50\n' +
                '61,2,0,0,0,0,0,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## 과제 3.4: 엔드포인트 삭제

다음의 3단계를 수행하여 엔드포인트를 정리할 수 있습니다. 먼저 엔드포인트를 삭제합니다. 그런 다음, 엔드포인트 구성을 삭제합니다. 마지막으로, 배포한 모델이 더 이상 필요하지 않다면 모델을 삭제합니다.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 마무리

축하합니다! SageMaker에서 SageMaker Python SDK를 사용하여 서버리스 엔드포인트를 생성했으며 해당 엔드포인트를 호출했습니다.

이 실습의 다음 과제에서는 비동기 추론을 사용하는 추론용 모델을 배포하는 과정을 중점적으로 살펴봅니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아와 **과제 4: 비동기 추론용으로 모델 배포**를 계속 진행합니다.